<a href="https://colab.research.google.com/github/usshaa/SMBDA/blob/main/C-5.3%3A%20RDD_Commands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### RDD Creation

1. **From a collection**:  

In [ ]:
from pyspark.sql import SparkSession
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext

In [ ]:
data = [1, 2, 10 ,3, 4, 5, 6]
rdd = sc.parallelize(data)
rdd.collect()

Out[5]: [1, 2, 10, 3, 4, 5, 6]

2. **From an external dataset** (e.g., text file):
   

In [ ]:
rdd0 = sc.textFile("/FileStore/tables/data.txt")
rdd0.collect()

Out[46]: ['Apache Spark has its architectural foundation in the resilient distributed dataset (RDD), a read-only multiset of data items distributed over a cluster of machines, that is maintained in a fault-tolerant way. The Dataframe API was released as an abstraction on top of the RDD, followed by the Dataset API. In Spark 1.x, the RDD was the primary application programming interface (API), but as of Spark 2.x use of the Dataset API is encouraged even though the RDD API is not deprecated. The RDD technology still underlies the Dataset API.',
 '',
 "Spark and its RDDs were developed in 2012 in response to limitations in the MapReduce cluster computing paradigm, which forces a particular linear dataflow structure on distributed programs: MapReduce programs read input data from disk, map a function across the data, reduce the results of the map, and store reduction results on disk. Spark's RDDs function as a working set for distributed programs that offers a (deliberately) restricted fo

### RDD Methods and Attributes

#### Transformation Methods

1. **`map(func)`**:
   - Applies `func` to each element and returns a new RDD.
   

In [ ]:
rdd_mapped = rdd.map(lambda x: x * 2)
rdd_mapped.collect()

Out[8]: [2, 4, 20, 6, 8, 10, 12]

2. **`filter(func)`**:
   - Applies `func` to each element and returns elements that satisfy the condition.   

In [ ]:
rdd_filtered = rdd.filter(lambda x: x % 2 == 0)
rdd_filtered.collect()

Out[9]: [2, 10, 4, 6]

3. **`flatMap(func)`**:
   - Similar to `map`, but flattens the result.
   

In [ ]:
rdd_flat_mapped = rdd.flatMap(lambda x: (x, x * 2))
rdd_flat_mapped.collect()

Out[10]: [1, 2, 2, 4, 10, 20, 3, 6, 4, 8, 5, 10, 6, 12]

4. **`union(otherRDD)`**:
   - Returns an RDD containing elements from both RDDs.
   

In [ ]:
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([4, 5, 6])
rdd_union = rdd1.union(rdd2)
rdd_union.collect()

Out[11]: [1, 2, 3, 4, 5, 6]

5. **`intersection(otherRDD)`**:
   - Returns an RDD with common elements between two RDDs.
   

In [ ]:
rdd_intersection = rdd1.intersection(rdd2)
rdd_intersection.collect()

Out[12]: []

6. **`distinct()`**:
   - Returns an RDD with distinct elements.
   

In [ ]:
rdd123 = sc.parallelize([4,4,5,5,6,10,20,30])
rdd123.collect()

Out[13]: [4, 4, 5, 5, 6, 10, 20, 30]

In [ ]:
rdd_distinct = rdd123.distinct()
rdd_distinct.collect()

Out[14]: [10, 4, 20, 5, 6, 30]

7. **`groupByKey()`** (deprecated, prefer `reduceByKey` or `aggregateByKey`):
   - Groups the values for each key in the RDD.
   

In [ ]:
rdd.collect()

Out[15]: [1, 2, 10, 3, 4, 5, 6]

In [ ]:
rdd_grouped = rdd.keyBy(lambda x: x % 2).groupByKey()
rdd_grouped.collect()

Out[16]: [(0, <pyspark.resultiterable.ResultIterable at 0x7f11e1bb4e50>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f11e1bb4f40>)]

8. **`reduceByKey(func)`**:
   - Reduces values for each key using the specified `func`.
   

In [ ]:
data = [1, 2, 17, 20 ,15, 4, 5, 5]
rdd = sc.parallelize(data)
rdd.collect()

Out[17]: [1, 2, 17, 20, 15, 4, 5, 5]

In [ ]:
rdd.collect()

Out[18]: [1, 2, 17, 20, 15, 4, 5, 5]

In [ ]:
rdd.filter(lambda x:x%3==0).collect()

Out[19]: [15]

In [ ]:
rdd.reduce(lambda x,y:x*y)

Out[20]: 1020000

In [ ]:
rdd.keyBy(lambda x: x).collect()

Out[21]: [(1, 1), (2, 2), (17, 17), (20, 20), (15, 15), (4, 4), (5, 5), (5, 5)]

In [ ]:
rdd_red = rdd.keyBy(lambda x: x).reduceByKey(lambda x, y: x + y).sortByKey()
rdd_red.collect()

Out[22]: [(1, 1), (2, 2), (4, 4), (5, 10), (15, 15), (17, 17), (20, 20)]

In [ ]:
rdd_reduced = rdd.keyBy(lambda x: x % 2).reduceByKey(lambda x, y: x + y*2)
rdd_reduced.collect()

Out[23]: [(0, 50), (1, 85)]

9. **`aggregateByKey(zeroValue, seqFunc, combFunc)`**:
   - Aggregates values for each key using sequence and combination functions.
   

In [ ]:
rdd.keyBy(lambda x: x % 2).collect()

Out[24]: [(1, 1), (0, 2), (1, 17), (0, 20), (1, 15), (0, 4), (1, 5), (1, 5)]

In [ ]:
seqOp = (lambda acc, value: acc + value)
combOp = (lambda acc1, acc2: acc1 + acc2)
rdd_aggregated = rdd.keyBy(lambda x: x % 2).aggregateByKey(0, seqOp, combOp)
rdd_aggregated.collect()

Out[25]: [(0, 26), (1, 43)]

### 10. **`sortByKey(ascending=True)`**:
    - Sorts RDD elements by key.
    

In [ ]:
rdd_sorted = rdd.keyBy(lambda x: x % 2).sortByKey()
rdd_sorted.collect()

Out[26]: [(0, 2), (0, 20), (0, 4), (1, 1), (1, 17), (1, 15), (1, 5), (1, 5)]

11. **`coalesce(numPartitions)`**:
    - Reduces the number of partitions to `numPartitions`.
    

In [ ]:
rdd_coalesced = rdd.coalesce(1)


12. **`repartition(numPartitions)`**:
    - Increases or decreases the number of partitions to `numPartitions`.
    

In [ ]:
rdd.getNumPartitions()

Out[28]: 8

In [ ]:
rdd_repartitioned.getNumPartitions()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1175556832608236>:1
----> 1 rdd_repartitioned.getNumPartitions()

NameError: name 'rdd_repartitioned' is not defined

In [ ]:
rdd = rdd.repartition(2)

In [ ]:
rdd_repartitioned = rdd.repartition(6)
rdd_repartitioned.collect()

Out[31]: [4, 5, 5, 17, 20, 1, 15, 2]

#### Action Methods

1. **`collect()`**:
   - Returns all elements from the RDD to the driver.
   

In [ ]:
collected_data = rdd.collect()


2. **`count()`**:
   - Returns the number of elements in the RDD.
   

In [ ]:
count = rdd.count()


3. **`take(n)`**:
   - Returns the first `n` elements from the RDD.
   

In [ ]:
first_elements = rdd.take(5)

4. **`reduce(func)`**:
   - Reduces the elements of the RDD using the specified associative and commutative `func`.
   

In [ ]:
rdd.reduce(lambda x, y: x + y)

Out[35]: 69

5. **`foreach(func)`**:
   - Applies `func` to each element of the RDD (usually for side effects).
   

In [ ]:
rdd.collect()

Out[36]: [17, 20, 15, 4, 5, 1, 2, 5]

In [ ]:
rdd.foreach(lambda x: print(x))

6. **`saveAsTextFile(path)`**:
   - Saves the RDD as a text file in the specified path.
   

In [ ]:
rdd.saveAsTextFile("output_directory1")

In [ ]:
rdd_repartitioned.saveAsTextFile("output_part")

#### Other Methods and Attributes

1. **`getNumPartitions()`**:
   - Returns the number of partitions in the RDD.
   

In [ ]:
num_partitions = rdd_repartitioned.getNumPartitions()
num_partitions

Out[40]: 6

2. **`toDF()`**:
   - Converts RDD to DataFrame (requires `pyspark.sql` module).
   

In [ ]:
# Convert RDD to DataFrame with a schema
df = rdd.map(lambda x: (x, )).toDF(["value"])

In [ ]:
# Show DataFrame
df.show()


+-----+
|value|
+-----+
|   17|
|   20|
|   15|
|    4|
|    5|
|    1|
|    2|
|    5|
+-----+



3. **`persist(storageLevel)`**:
   - Persists RDD with specified storage level (e.g., MEMORY_ONLY, MEMORY_AND_DISK).
   

In [ ]:
from pyspark import StorageLevel
rdd.persist(StorageLevel.MEMORY_AND_DISK)


Out[43]: MapPartitionsRDD[71] at coalesce at NativeMethodAccessorImpl.java:0

4. **`unpersist()`**:
   - Removes RDD from memory/disk cache.
   

In [ ]:
rdd.unpersist()

Out[44]: MapPartitionsRDD[71] at coalesce at NativeMethodAccessorImpl.java:0

5. **`is_cached()`**:
   - Returns `True` if the RDD is cached.
   

In [ ]:
######
is_cached = rdd.is_cached
is_cached

Out[45]: False

### Notes:
- **Lazy Evaluation**: Transformations on RDDs are lazily evaluated, meaning they are not computed until an action is called.
- **Immutability**: RDDs are immutable once created; transformations create new RDDs rather than modifying existing ones.
- **Context**: `sc` in the examples refers to the SparkContext object (`sc = SparkContext(...)`) which is typically created when starting a Spark application.

This cheatsheet covers a wide range of RDD methods, transformations, actions, and attributes in Apache Spark using Python (PySpark). Adjustments may be needed for specific use cases or version differences in Spark.